# $unwind 的基本操作

In [ ]:
# finding the top rated genres per year from 2010 to 2015...
db.movies.aggregate([
    {
        $match: {
            "imdb.rating": { $gt: 0},
            year: { $gte: 2010, $lte: 2015},
            runtime: { $gte: 90 }
        }
    },
    { $unwind: "$genres" },
    {
        $group: {
            _id: { year: "$year", genre: "$genres" },
            average_rating: { $avg: "$imdb.rating" }
        }
    },
    { $sort: { "_id.year": -1, average_rating: -1 }}

]).pretty()

In [ ]:
# unfortunately we got too many results per year back. Rather than peform some
# other complex grouping and matching, we just append a simple group and sort
# stage, taking advantage of the fact the documents are in the order we want
db.movies.aggregate([
    {
        $match: {
            "imdb.rating": { $gt: 0 },
            year: { $gte: 2010, $lte: 2015 },
            runtime: { $gte: 90 }
        }
    },
    { $unwind: "$genres" },
    {
        $group: {
            _id: { year: "$year", genre: "$genres" },
            average_rating: { $avg: "$imdb.rating" }
        }
    },
    { $sort: { "_id.year": -1, average_rating: -1 }},
    {
        $group: {
            _id: "$_id.year",
            genre: { $first: "$_id.genre" },
            average_rating: { $first: "$average_rating" }
        }
    },
    { $sort: { _id: -1 }}
    
]).pretty()

### 範例01

In [ ]:
# 參考解答
db.movies.aggregate([
    {
        $match: {
            languages: { $in: [ "English" ]}
        }
    },
    { $unwind: "$cast"},
    {
        $group: {
            _id: "$cast",
            numFilms: { $sum: 1 },
            average: { $avg: "$imdb.rating" }
        }
    },
    {
        $project: {
            _id: 1,
            numFilms: 1,
            average: { $trunc: [ "$average", 1 ]}
        }
    },
    { $sort: { numFilms: -1 }},
    { $limit: 1 }
])

In [ ]:
# 參考解答
db.movies.aggregate([
  {
    $match: {
      languages: "English"
    }
  },
  {
    $project: { _id: 0, cast: 1, "imdb.rating": 1 }
  },
  {
    $unwind: "$cast"
  },
  {
    $group: {
      _id: "$cast",
      numFilms: { $sum: 1 },
      average: { $avg: "$imdb.rating" }
    }
  },
  {
    $project: {
      numFilms: 1,
      average: {
        $divide: [{ $trunc: { $multiply: ["$average", 10] } }, 10]
      }
    }
  },
  {
    $sort: { numFilms: -1 }
  },
  {
    $limit: 1
  }
])